# AWS DevDay Seoul 2019
## 모두를 위한 컴퓨터 비전 딥러닝 툴킷, GluonCV 따라하기
## Lab 2.1 GluonCV Model Zoo의  Pretrained 모델 사용하기

<!-- This notebook is based on: https://github.com/dmlc/web-data/blob/master/gluoncv/tutorials/AI_In_Practice/AI_in_practice_gluoncv_0622.ipynb -->

노트북을 처음 로딩할 때, Kernel로 **conda_mxnet_p36** 을 선택합니다.

### 랩 순서

1. Image Classification
2. Object Detection
3. (Option) Semantic Segmentation

#### GluonCV와 필요한 python 패키지를 설치합니다.
GluonCV의 `model_zoo`와 `utils` 패키지에 대해서는 아래 링크를 참조하세요.
- `model_zoo`: [https://gluon-cv.mxnet.io/model_zoo/index.html](https://gluon-cv.mxnet.io/model_zoo/index.html)
- `utils`: [https://gluon-cv.mxnet.io/api/utils.html](https://gluon-cv.mxnet.io/api/utils.html)

In [ ]:
# 최초 실행시 GPU(p2/p3) instance에서는 아래 코드로 gluoncv 패키지를 설치하세요.
!pip install --upgrade mxnet-cu100mkl gluoncv
# CPU(c4/c5/m4/m5/t2/t3) instance에서는 아래 코드로 gluoncv 패키지를 설치하세요.
#!pip install --upgrade mxnet-mkl gluoncv

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import mxnet as mx
import numpy as np
import gluoncv as gcv
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from mxnet import nd, image
from mxnet.gluon.data.vision import transforms
from gluoncv.model_zoo import get_model
from gluoncv.utils import viz, download

# Task 1 - Image Classification

Image classification은 컴퓨터 비전에서 가장 기본이 되는 구성 요소라고 할 수 있습니다. 아래에서는 `GluonCV` [Model Zoo](https://gluon-cv.mxnet.io/model_zoo/index.html)의 미리 훈련된(pretrained) 모델을 사용해서 간단한 image classification 작업을 해보겠습니다.

먼저, 데이터를 로드합니다.

In [ ]:
plt.rcParams['figure.figsize'] = (15, 9)

img = image.imread('mt_baker.jpg')
viz.plot_image(img)
plt.show()

`GluonCV Model Zoo`가 지원하는 classification 모델의 전체 목록은 [Model Zoo > Classification](https://gluon-cv.mxnet.io/model_zoo/classification.html) 링크에서 확인할 수 있습니다.

아래 한 줄 코드로 pretrained `Resnet50_v1` 모델을 로드합니다.

In [ ]:
net = get_model('resnet50_v1', pretrained=True)

이미지를 모델에 입력하기 전에, 이미지의 normalization 단계가 필요합니다.

In [ ]:
transform_fn = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

img = transform_fn(img)

Prediction을 위해 아래 코드를 실행합니다.

In [ ]:
pred = net(img.expand_dims(0))

Top 5의 예측 결과를 확인해 봅니다 (`ImageNet`의 label 정보 기준).

In [ ]:
classes_fname = download('https://raw.githubusercontent.com/hetong007/gluon-cv/master/scripts/classification/imagenet/imagenet_labels.txt',
                         'imagenet_labels.txt')
with open('imagenet_labels.txt', 'r') as f:
    class_names = [l.strip('\n') for l in f.readlines()]

topK = 5
ind = nd.topk(pred, k=topK).astype('int')[0]
for i in range(topK):
    print('[%s], with probability %.1f%%'%
         (class_names[ind[i].asscalar()], nd.softmax(pred)[0][ind[i]].asscalar()*100))

`ImageNet`의 [label 집합](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a)에는 놀랍게도 'mountain'이 없습니다. 대신 시각적으로 가장 유사한 'volcano'로 분류가 되었습니다.

다른 이미지로 다시 한번 image classification 작업을 해보겠습니다. 입력 이미지를 먼저 읽어옵니다.

In [ ]:
img = image.imread('cr7.jpg')
viz.plot_image(img)
plt.show()

모델은 다시 읽어올 필요가 없습니다. 입력 이미지의 normalization을 거쳐 prediction을 해봅니다.

In [ ]:
img = transform_fn(img)
pred = net(img.expand_dims(0))
ind = nd.topk(pred, k=topK).astype('int')[0]
for i in range(topK):
    print('[%s], with probability %.1f%%'%
         (class_names[ind[i].asscalar()], nd.softmax(pred)[0][ind[i]].asscalar()*100))

`ImageNet`의 [label 집합](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a)에는 'soccer'도 없습니다. 대신 가장 가까운 'soccer_ball'으로 분류가 되었습니다.

# Task 2 - Object Detection

Image classification은 가장 중요한 객체를 인식하는데 사용될 수 있습니다. 하지만 인식할 대상이 여럿이라면 어떻게 해야할까요?

In [ ]:
x, img = gcv.data.transforms.presets.ssd.load_test('cr7.jpg', short=512)
viz.plot_image(img)
plt.show()

이번에는 object detection 모델 중에서 `SSD`를 사용해보겠습니다.

`GluonCV Model Zoo`가 지원하는 detection 모델의 전체 목록은 [Model Zoo > Detection](https://gluon-cv.mxnet.io/model_zoo/detection.html) 링크에서 확인할 수 있습니다.

In [ ]:
net = get_model('ssd_512_resnet50_v1_coco', pretrained=True)

이제 예측해보겠습니다. 출력되는 variable 3개는 각각 예측된 class ID, confidence score, bounding box입니다.


In [ ]:
class_IDs, scores, bounding_boxs = net(x)
viz.plot_bbox(img, bounding_boxs[0], scores[0], class_IDs[0], class_names=net.classes)
plt.show()

이번에는 같은 모델로 이미지 속의 사람 수를 세어 보겠습니다.

In [ ]:
x, img = gcv.data.transforms.presets.ssd.load_test('crowd.png', short=512)
viz.plot_image(img)
plt.show()

In [ ]:
class_IDs, scores, bounding_boxs = net(x)
person_ind = [i for i, cls in enumerate(net.classes) if cls == 'person']
ind = np.nonzero(class_IDs[0].asnumpy() == person_ind)[0]

new_class_IDs = class_IDs[0][ind]
new_scores = scores[0][ind]
new_bounding_boxs = bounding_boxs[0][ind]

viz.plot_bbox(img, new_bounding_boxs, new_scores, new_class_IDs, class_names=net.classes)
plt.show()

print('There are %d people in this photo.'%(len(ind)))

이번에는 자율주행 시나리오에 적용해 보겠습니다.

In [ ]:
x, img = gcv.data.transforms.presets.ssd.load_test('streetview_amazon.png', short=512)
viz.plot_image(img)
plt.show()

In [ ]:
class_IDs, scores, bounding_boxs = net(x)
viz.plot_bbox(img, bounding_boxs[0], scores[0], class_IDs[0], class_names=net.classes)
plt.show()

# (Option) Task 3 - Semantic Segmentation

Semantic segmentation은 각각의 픽셀마다 class를 예측하는 문제입니다.

In [ ]:
img = image.imread('streetview_amazon.png')
viz.plot_image(img)
plt.show()

In [ ]:
transform_fn = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([.485, .456, .406], [.229, .224, .225])
])
img = transform_fn(img)
img = img.expand_dims(0)

이번에는 semantic segmentation 모델 중에서 `ADE20K dataset`으로 pretrained된 `FCN`을 사용해보겠습니다.

`GluonCV Model Zoo`가 지원하는 segmentation 모델의 전체 목록은 [Model Zoo > Segmentation](https://gluon-cv.mxnet.io/model_zoo/segmentation.html) 링크에서 확인할 수 있습니다.

In [ ]:
net = get_model('fcn_resnet50_ade', pretrained=True)

In [ ]:
output = net.demo(img)
pred = mx.nd.squeeze(mx.nd.argmax(output, 1)).asnumpy()

In [ ]:
mask = viz.get_color_pallete(pred, 'ade20k')
mask.save('output_amazon.png')
mmask = mpimg.imread('output_amazon.png')
plt.imshow(mmask)
plt.show()

또 다른 이미지로 semantic segmentation을 해보겠습니다.

In [ ]:
img = image.imread('streetview_paloalto.png')
viz.plot_image(img)
plt.show()

In [ ]:
img = transform_fn(img)
img = img.expand_dims(0)

In [ ]:
output = net.demo(img)
pred = mx.nd.squeeze(mx.nd.argmax(output, 1)).asnumpy()

In [ ]:
mask = viz.get_color_pallete(pred, 'ade20k')
mask.save('output_paloalto.png')
mmask = mpimg.imread('output_paloalto.png')
plt.imshow(mmask)
plt.show()

지금까지 [GluonCV Model Zoo](https://gluon-cv.mxnet.io/model_zoo/index.html)의 다양한 pretrained model에 대해 알아봤습니다. `Pose estimation`, `action recognition` 등의 다른 모델에 대해서도 체크해 보세요!